In [2]:
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 4, Finished, Available)

In [1]:
df = spark.sql("SELECT * FROM Spotify_Lake.raye_popular_tracks LIMIT 1000")
display(df)

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, d253a775-55fd-48da-a32c-31b62d3d7476)

In [3]:
# schema for tracks dimension table
DeltaTable.createIfNotExists(spark) \
          .tableName('Gold_Spotify_Lake.tracks_gold') \
          .addColumn('TrackID', StringType()) \
          .addColumn('Track_Name', StringType()) \
          .addColumn('Track_Popularity', IntegerType()) \
          .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 5, Finished, Available)

In [4]:
# dataframe for tracks_gold
dftracks_gold = df.dropDuplicates(['tracks_id']).select(col('tracks_id').alias('TrackID'), 
                                                        col('tracks_name').alias('Track_Name'), 
                                                        col('tracks_popularity').alias('Track_Popularity')
                                                        )


StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 6, Finished, Available)

In [5]:
# updates to the tracks_gold

deltaTable = DeltaTable.forPath(spark, 'abfss://Spotify@onelake.dfs.fabric.microsoft.com/Gold_Spotify_Lake.Lakehouse/Tables/tracks_gold')

dfUpdates = dftracks_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.TrackID = updates.TrackID'
  ) \
 .whenNotMatchedInsert(values =
    {     
     'TrackID': 'updates.TrackID',
     'Track_Name': 'updates.Track_Name',
     'Track_Popularity': 'updates.Track_Popularity',
    }
  ) \
  .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 7, Finished, Available)

In [6]:
# schema for artist dimension table
DeltaTable.createIfNotExists(spark) \
          .tableName('Gold_Spotify_Lake.artist_gold') \
          .addColumn('ArtistID', StringType()) \
          .addColumn('Artist_Name', StringType()) \
          .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 8, Finished, Available)

In [7]:
# dataframe for artist_gold
dfartist_gold = df.dropDuplicates(['tracks_artists_id']).select(col('tracks_artists_id').alias('ArtistID'), 
                                                        col('tracks_artists_name').alias('Artist_Name')
                                                        )


StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 9, Finished, Available)

In [8]:
# updates to the artist_gold

deltaTable = DeltaTable.forPath(spark, 'abfss://Spotify@onelake.dfs.fabric.microsoft.com/Gold_Spotify_Lake.Lakehouse/Tables/artist_gold')

dfUpdates = dfartist_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.ArtistID = updates.ArtistID'
  ) \
 .whenNotMatchedInsert(values =
    {     
     'ArtistID': 'updates.ArtistID',
     'Artist_Name': 'updates.Artist_Name',
    }
  ) \
  .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 10, Finished, Available)

In [9]:
# schema for album dimension table
DeltaTable.createIfNotExists(spark) \
          .tableName('Gold_Spotify_Lake.album_gold') \
          .addColumn('AlbumID', StringType()) \
          .addColumn('Album_Name', StringType()) \
          .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 11, Finished, Available)

In [10]:
# dataframe for album_gold
dfalbum_gold = df.dropDuplicates(['tracks_album_id']).select(col('tracks_album_id').alias('AlbumID'), 
                                                        col('tracks_album_name').alias('Album_Name')
                                                        )

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 12, Finished, Available)

In [11]:
# updates to the album_gold

deltaTable = DeltaTable.forPath(spark, 'abfss://Spotify@onelake.dfs.fabric.microsoft.com/Gold_Spotify_Lake.Lakehouse/Tables/album_gold')

dfUpdates = dfalbum_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.AlbumID = updates.AlbumID'
  ) \
 .whenNotMatchedInsert(values =
    {     
     'AlbumID': 'updates.AlbumID',
     'Album_Name': 'updates.Album_Name',
    }
  ) \
  .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 13, Finished, Available)

In [12]:
# schema for raye_gold fact table
DeltaTable.createIfNotExists(spark) \
          .tableName('Gold_Spotify_Lake.raye_gold') \
          .addColumn('ArtistID', StringType()) \
          .addColumn('AlbumID', StringType()) \
          .addColumn('TrackID', StringType()) \
          .addColumn('Track_Duration_ms', IntegerType()) \
          .addColumn('Track_Explicit', BooleanType()) \
          .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 14, Finished, Available)

In [13]:
tracks_gold_temp = spark.read.table('Gold_Spotify_Lake.tracks_gold')
artist_gold_temp = spark.read.table('Gold_Spotify_Lake.artist_gold')

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 15, Finished, Available)

In [14]:
# create factholidays_gold dataframe
dfraye_gold = df.alias('df1') \
    .join(tracks_gold_temp.alias('df2'), (col('df1.tracks_id') == col('df2.TrackID')), 'left') \
    .join(artist_gold_temp.alias('df3'), (col('df1.tracks_album_artists_id') == col('df3.ArtistID')), 'left') \
    .select(
        col('df2.TrackID').alias('TrackID'),
        col('df1.tracks_explicit').alias('Track_Explicit'),
        col('df3.ArtistID').alias('ArtistID'),
        col('df1.tracks_album_id').alias('AlbumID'),
        col('df1.tracks_duration_ms').alias('Track_Duration_ms')
    ) \
    .orderBy(col('df3.ArtistID'))

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 16, Finished, Available)

In [15]:
display(dfraye_gold)

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, a1643552-7d1d-4675-a057-4f2e0c56c649)

In [17]:
# updates to the raye_gold

deltaTable = DeltaTable.forPath(spark, 'abfss://Spotify@onelake.dfs.fabric.microsoft.com/Gold_Spotify_Lake.Lakehouse/Tables/raye_gold')

dfUpdates = dfraye_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.AlbumID = updates.AlbumID AND silver.ArtistID = updates.ArtistID AND silver.TrackID = updates.TrackID'
  ) \
 .whenNotMatchedInsert(values =
    { 
     'ArtistID': 'updates.ArtistID',       
     'AlbumID': 'updates.AlbumID',
     'TrackID': 'updates.TrackID',
     'Track_Duration_ms': 'updates.Track_Duration_ms',
     'Track_Explicit': 'updates.Track_Explicit'
    }
  ) \
  .execute()

StatementMeta(, ecb73ad6-f877-4fdc-8654-83191c078792, 19, Finished, Available)